# FDA Project

Installing specialized packages: webdriver-manager, selenium, and wordcloud

In [1]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium

In [3]:
conda install -c https://conda.anaconda.org/conda-forge wordcloud

Note: you may need to restart the kernel to use updated packages.
Solving environment: ...working... done

# All requested packages already installed.




In [4]:
#Importing of useful packages
import pandas as pd
import re
import numpy as np
import json
import datetime
from bs4 import BeautifulSoup as bs
import requests
import re
import urllib
import time
import os
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
#% matplotlib inline
from wordcloud import WordCloud
from collections import Counter


URL = "https://open.fda.gov/data/downloads/"

In [5]:
#The FDA website does not load the downloadable files unless you scroll to that area of the page first
#Web-Scraping involves using the Selenium webdriver to open the site with Chrome, navigate to the
    #needed area, and hit the correct buttons at the correct time

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import selenium.common.exceptions
from selenium import webdriver
import time

from selenium.webdriver.support.wait import WebDriverWait

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.google.com")

driver.get(URL)
driver.maximize_window()

time.sleep(1)

button1 = driver.find_element(By.CLASS_NAME, "button.bg-primary.clr-white")
button1.click()

time.sleep(1)
 
element_link=WebDriverWait(driver, 10).until(EC.presence_of_element_located(
   (By.XPATH, '//*[@id="Medical Device Event"]')))

driver.execute_script("arguments[0].scrollIntoView(true)", element_link)

time.sleep(1)

button2 = driver.find_element(By.XPATH, '//*[@id="Medical Device Event"]/section/button')
button2.click()

time.sleep(1)

html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
#print (html)

driver.close()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\Allen\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [6]:
#Snip HTML to just the portion in question
pattern = '1991(.*?)<li id="Medical Device PMA">'
substring = re.search(pattern, html).group(1)

In [7]:
#Itemize links into array
import lxml.html

url_list = lxml.html.fromstring(substring)
url_list = url_list.xpath('//a/@href')
#print(newlist)

In [8]:
start_year = 2020
end_year = 2020

year_list = list(range(start_year, end_year+1))

index_to_download = []

for meh in year_list:
    for bleh in range(0, len(url_list)):
        if str(meh) in url_list[bleh]:
            index_to_download.append(url_list.index(url_list[bleh]))

index_count = len(index_to_download)

In [9]:
#Follow links in array to download/process ZIPs

#Specify Download Path
path = 'C:/Users/Allen/Documents/FDA'
#pathinverse = 'C:\Users\Allen\Documents\FDA'
import requests, zipfile
from io import BytesIO

#Filter Data by Project Code, use "All" to include all data
pcode = ["KDK","KDL","KDN","LLJ","MFU","MFV","MSB","PGR","PHO","PIN","QBA","QEJ"]
#pcode = ["All"]

#Run loop, opening JSONs
loopnumber = 0
datamain = ""
for snuh in index_to_download:
    print('Download ' + (str(loopnumber+1)) + " of " + (str(index_count)) + " started ")
    url = url_list[snuh]
    import requests, zipfile
    req = requests.get(url)
    print('Download ' + (str(loopnumber+1))+ " completed ")
    zipfile = zipfile.ZipFile(BytesIO(req.content))
    #filename = "FDA" + str(snuh+1)
    filename = "FDAdata.json"
    for i, f in enumerate(zipfile.filelist):
        f.filename = filename.format(i)
        zipfile.extract(f)
    print('File ' + (str(loopnumber+1))+ ' extracted')
    data = json.load(open(r'C:\Users\Allen\Documents\FDA\FDAdata.json'))
    data = data["results"]
    datamain = data
    if loopnumber == 0:
        print('Creating Dataframe with JSON ' + (str(loopnumber+1)))
        dfmain = pd.json_normalize(data,
                  record_path = "device",
                  meta = ["report_number","report_source_code","date_received","event_type","type_of_report","mdr_text"],
                  record_prefix = "_",
                  errors = "ignore")
        if pcode[0] != "All":
            dfmain = dfmain[dfmain._device_report_product_code.isin(pcode)]
        print('Dataframe Created')
    else:
        print('Appending Dataframe with JSON ' + (str(loopnumber+1)))
        dfnew = pd.json_normalize(data,
                  record_path = "device",
                  meta = ["report_number","report_source_code","date_received","event_type","type_of_report","mdr_text"],
                  record_prefix = "_",
                  errors = "ignore")
        if pcode[0] != "All":
            dfnew = dfnew[dfnew._device_report_product_code.isin(pcode)]
        dfmain = pd.concat([dfmain, dfnew])
        #dfmain.append(dfnew)
        print('JSON ' + (str(loopnumber+1)) + ' appended')
        
    os.remove(path + "/" + filename)
    loopnumber = loopnumber + 1
print("Dataframe ready")
dfmain.head()

Download 1 of 17 started 
Download 1 completed 
File 1 extracted
Creating Dataframe with JSON 1
Dataframe Created
Download 2 of 17 started 
Download 2 completed 
File 2 extracted
Appending Dataframe with JSON 2
JSON 2 appended
Download 3 of 17 started 
Download 3 completed 
File 3 extracted
Appending Dataframe with JSON 3
JSON 3 appended
Download 4 of 17 started 
Download 4 completed 
File 4 extracted
Appending Dataframe with JSON 4
JSON 4 appended
Download 5 of 17 started 
Download 5 completed 
File 5 extracted
Appending Dataframe with JSON 5
JSON 5 appended
Download 6 of 17 started 
Download 6 completed 
File 6 extracted
Appending Dataframe with JSON 6
JSON 6 appended
Download 7 of 17 started 
Download 7 completed 
File 7 extracted
Appending Dataframe with JSON 7
JSON 7 appended
Download 8 of 17 started 
Download 8 completed 
File 8 extracted
Appending Dataframe with JSON 8
JSON 8 appended
Download 9 of 17 started 
Download 9 completed 
File 9 extracted
Appending Dataframe with JSON 

_device_event_key _implant_flag _date_removed_flag  \
98596                                                      
17151                                                      
42763                                                      
70387                                                      
20329                                                      

      _device_sequence_number _date_received  \
98596                       1       20200206   
17151                       1       20200227   
42763                       1       20200206   
70387                       1       20200206   
20329                       1       20200320   

                              _brand_name      _generic_name  \
98596  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
17151  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
42763  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
70387  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
20329  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   

               _manufacturer_d_name _manufacturer_d_address_1  \
98596  ORGAN RECOVERY SYSTEMS, INC.            1 PIERCE PLACE   
17151  ORGAN RECOVERY SYSTEMS, INC.            1 PIERCE PLACE   
42763  ORGAN RECOVERY SYSTEMS, INC.            1 PIERCE PLACE   
70387  ORGAN RECOVERY SYSTEMS, INC.            1 PIERCE PLACE   
20329  ORGAN RECOVERY SYSTEMS, INC.            1 PIERCE PLACE   

      _manufacturer_d_address_2  ... _openfda.device_class  \
98596                SUITE 475W  ...                     2   
17151                SUITE 475W  ...                     2   
42763                SUITE 475W  ...                     2   
70387                SUITE 475W  ...                     2   
20329                SUITE 475W  ...                     2   

      _expiration_date_of_device _openfda.registration_number  \
98596                   20210318                          NaN   
17151                   20210318                 [3004068499]   
42763                   20210424                          NaN   
70387                   20210318                 [3004068499]   
20329                   20211029                 [3004068499]   

      _openfda.fei_number          report_number   report_source_code  \
98596                 NaN  3004068499-2020-00003  Manufacturer report   
17151        [3004068499]  3004068499-2020-00004  Manufacturer report   
42763                 NaN  3004068499-2020-00001  Manufacturer report   
70387        [3004068499]  3004068499-2020-00002  Manufacturer report   
20329        [3004068499]  3004068499-2020-00005  Manufacturer report   

      date_received   event_type                  type_of_report  \
98596      20200206  Malfunction  [Initial submission, Followup]   
17151      20200227  Malfunction            [Initial submission]   
42763      20200206  Malfunction  [Initial submission, Followup]   
70387      20200206  Malfunction  [Initial submission, Followup]   
20329      20200320  Malfunction            [Initial submission]   

                                                mdr_text  
98596  [{'mdr_text_key': '221303083', 'text_type_code...  
17151  [{'mdr_text_key': '224587763', 'text_type_code...  
42763  [{'mdr_text_key': '196742172', 'text_type_code...  
70387  [{'mdr_text_key': '221303162', 'text_type_code...  
20329  [{'mdr_text_key': '222081101', 'text_type_code...  

[5 rows x 40 columns]

In [10]:
#dfmain.info()

#Keep only relevant columns
dfmain = dfmain[["_device_report_product_code","_brand_name","_generic_name","_manufacturer_d_name","type_of_report","report_number","report_source_code",
                 "date_received","event_type","mdr_text"]]

#Rename columns
dfmain.columns = ["product_code","brand_name","generic_name","manufacturer_name","type_of_report","report_number",
                        "report_source_code","date_received","event_type","mdr_text"]

#Update date column to date format
dfmain["date_received"] = pd.to_datetime(dfmain["date_received"])

#Remove brackets from type of report column
dfmain['type_of_report'] = dfmain['type_of_report'].str.join(', ')

#Update MDR Text to only show the text narrative items--also lowercase the text
newmdr = []
for crag in dfmain["mdr_text"]:
    newmdr.append(''.join(re.findall("'text': .+?}",str(crag))).translate(str.maketrans('', '', string.punctuation)).replace("text"," - ")[4:])

dfmain["mdr_text"] = [x.lower() for x in newmdr]
dfmain.head()

product_code                          brand_name       generic_name  \
98596          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
17151          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
42763          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
70387          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
20329          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   

                  manufacturer_name                type_of_report  \
98596  ORGAN RECOVERY SYSTEMS, INC.  Initial submission, Followup   
17151  ORGAN RECOVERY SYSTEMS, INC.            Initial submission   
42763  ORGAN RECOVERY SYSTEMS, INC.  Initial submission, Followup   
70387  ORGAN RECOVERY SYSTEMS, INC.  Initial submission, Followup   
20329  ORGAN RECOVERY SYSTEMS, INC.            Initial submission   

               report_number   report_source_code date_received   event_type  \
98596  3004068499-2020-00003  Manufacturer report    2020-02-06  Malfunction   
17151  3004068499-2020-00004  Manufacturer report    2020-02-27  Malfunction   
42763  3004068499-2020-00001  Manufacturer report    2020-02-06  Malfunction   
70387  3004068499-2020-00002  Manufacturer report    2020-02-06  Malfunction   
20329  3004068499-2020-00005  Manufacturer report    2020-03-20  Malfunction   

                                                mdr_text  
98596  investigation for this complaint is currently ...  
17151  patient information does not apply the device ...  
42763  investigation for this complaint is currently ...  
70387  investigation for this complaint is currently ...  
20329  patient information does not apply the device ...

In [11]:
#dfmain = pd.read_csv("data.csv", encoding = 'unicode_escape')

#Remove stop words in MDR Text--we must remove null rows before this step can be done
stop_words = set(stopwords.words('english'))

#Add additional common words into stopwords
stop_words.update(["investigation","still","progress","complete","supplemental","report","filed","device","returned","reported",
                  "-","due","failure","failed","may","ensure","assures","around","met","reports","number","per","dated"])

dfmain = dfmain[dfmain['mdr_text'].notnull()]

dfmain['mdr_text'] = dfmain['mdr_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

dfmain.head()

product_code                          brand_name       generic_name  \
98596          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
17151          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
42763          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
70387          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
20329          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   

                  manufacturer_name                type_of_report  \
98596  ORGAN RECOVERY SYSTEMS, INC.  Initial submission, Followup   
17151  ORGAN RECOVERY SYSTEMS, INC.            Initial submission   
42763  ORGAN RECOVERY SYSTEMS, INC.  Initial submission, Followup   
70387  ORGAN RECOVERY SYSTEMS, INC.  Initial submission, Followup   
20329  ORGAN RECOVERY SYSTEMS, INC.            Initial submission   

               report_number   report_source_code date_received   event_type  \
98596  3004068499-2020-00003  Manufacturer report    2020-02-06  Malfunction   
17151  3004068499-2020-00004  Manufacturer report    2020-02-27  Malfunction   
42763  3004068499-2020-00001  Manufacturer report    2020-02-06  Malfunction   
70387  3004068499-2020-00002  Manufacturer report    2020-02-06  Malfunction   
20329  3004068499-2020-00005  Manufacturer report    2020-03-20  Malfunction   

                                                mdr_text  
98596  complaint currently underway yet completed fol...  
17151  patient information apply patient contact capa...  
42763  complaint currently underway yet completed fol...  
70387  complaint currently underway yet completed fol...  
20329  patient information apply patient contact outc...

In [12]:
#dfmain = pd.read_csv("data.csv", encoding = 'unicode_escape')

no_dup = []

#Following function found from internet
def remove_duplicates(input):
 
    # split input string separated by space
    input = input.split(" ")
 
    # now create dictionary using counter method
    # which will have strings as key and their
    # frequencies as value
    UniqW = Counter(input)
 
    # joins two adjacent elements in iterable way
    s = " ".join(UniqW.keys())
    return (s)

dfmain['mdr_text_nodup'] = dfmain['mdr_text'].apply(remove_duplicates)

dfmain['mdr_text_nodup'] = dfmain['mdr_text_nodup'].map(str)

dfmain.head()

product_code                          brand_name       generic_name  \
98596          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
17151          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
42763          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
70387          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
20329          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   

                  manufacturer_name                type_of_report  \
98596  ORGAN RECOVERY SYSTEMS, INC.  Initial submission, Followup   
17151  ORGAN RECOVERY SYSTEMS, INC.            Initial submission   
42763  ORGAN RECOVERY SYSTEMS, INC.  Initial submission, Followup   
70387  ORGAN RECOVERY SYSTEMS, INC.  Initial submission, Followup   
20329  ORGAN RECOVERY SYSTEMS, INC.            Initial submission   

               report_number   report_source_code date_received   event_type  \
98596  3004068499-2020-00003  Manufacturer report    2020-02-06  Malfunction   
17151  3004068499-2020-00004  Manufacturer report    2020-02-27  Malfunction   
42763  3004068499-2020-00001  Manufacturer report    2020-02-06  Malfunction   
70387  3004068499-2020-00002  Manufacturer report    2020-02-06  Malfunction   
20329  3004068499-2020-00005  Manufacturer report    2020-03-20  Malfunction   

                                                mdr_text  \
98596  complaint currently underway yet completed fol...   
17151  patient information apply patient contact capa...   
42763  complaint currently underway yet completed fol...   
70387  complaint currently underway yet completed fol...   
20329  patient information apply patient contact outc...   

                                          mdr_text_nodup  
98596  complaint currently underway yet completed fol...  
17151  patient information apply contact capab4 initi...  
42763  complaint currently underway yet completed fol...  
70387  complaint currently underway yet completed fol...  
20329  patient information apply contact outcomes att...

In [13]:
#Implement Word Cloud
word_cloud = WordCloud(collocations = False, background_color = 'white').generate(str(dfmain["mdr_text_nodup"]))
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('foo.png')
plt.show()

In [14]:
#Create new date field with the days removed, for visualizations

#dfmain = pd.read_csv("data.csv", encoding = 'unicode_escape')
#dfmain['mdr_text_nodup'] = dfmain['mdr_text_nodup'].map(str)

dfmain["year"] = pd.to_datetime(dfmain["date_received"], format = '%Y-%m-%d').dt.year
dfmain["month"] = pd.to_datetime(dfmain["date_received"], format = '%Y-%m-%d').dt.month

dfmain["plaindate"] = pd.to_datetime(dfmain[['year', 'month']].assign(DAY=1))

dfmain = dfmain.drop(["year","month"], 1)

dfmain.head()

#dfmain.to_csv(r'data_organ.csv', index = False)

C:\Users\Allen\AppData\Local\Temp\ipykernel_25376\1247217391.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dfmain = dfmain.drop(["year","month"], 1)


In [18]:
dfmain.head()

product_code                          brand_name       generic_name  \
98596          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
17151          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
42763          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
70387          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   
20329          KDN  LIFEPORT KIDNEY TRANSPORTER SYSTEM  PERFUSION CIRCUIT   

                  manufacturer_name                type_of_report  \
98596  ORGAN RECOVERY SYSTEMS, INC.  Initial submission, Followup   
17151  ORGAN RECOVERY SYSTEMS, INC.            Initial submission   
42763  ORGAN RECOVERY SYSTEMS, INC.  Initial submission, Followup   
70387  ORGAN RECOVERY SYSTEMS, INC.  Initial submission, Followup   
20329  ORGAN RECOVERY SYSTEMS, INC.            Initial submission   

               report_number   report_source_code date_received   event_type  \
98596  3004068499-2020-00003  Manufacturer report    2020-02-06  Malfunction   
17151  3004068499-2020-00004  Manufacturer report    2020-02-27  Malfunction   
42763  3004068499-2020-00001  Manufacturer report    2020-02-06  Malfunction   
70387  3004068499-2020-00002  Manufacturer report    2020-02-06  Malfunction   
20329  3004068499-2020-00005  Manufacturer report    2020-03-20  Malfunction   

                                                mdr_text  \
98596  complaint currently underway yet completed fol...   
17151  patient information apply patient contact capa...   
42763  complaint currently underway yet completed fol...   
70387  complaint currently underway yet completed fol...   
20329  patient information apply patient contact outc...   

                                          mdr_text_nodup  plaindate  
98596  complaint currently underway yet completed fol... 2020-02-01  
17151  patient information apply contact capab4 initi... 2020-02-01  
42763  complaint currently underway yet completed fol... 2020-02-01  
70387  complaint currently underway yet completed fol... 2020-02-01  
20329  patient information apply contact outcomes att... 2020-03-01

In [15]:
#Create new dataframe--list of unique words in data, separated by month
#dfmain = pd.read_csv("data_vent.csv", encoding = 'unicode_escape')
dfmain['mdr_text_nodup'] = dfmain['mdr_text_nodup'].map(str)

dfword = pd.DataFrame()

#Create unique list of dates to be used in the for loop
datelist = dfmain["plaindate"].unique()

#Loop to create dataframe of unique words and the number of times they each appear
wordnumbers = []
for squid in datelist:
    wordnumbers = Counter(' '.join(map(lambda l: ''.join(l), dfmain.loc[dfmain["plaindate"] == squid]["mdr_text_nodup"])).split(" ")) 
    dftemp = pd.DataFrame.from_dict(wordnumbers, orient='index').reset_index()
    dftemp["Date"] = squid
    dfword = pd.concat([dfword, dftemp])
    
dfword.columns = ["word","counts","date"]

#dfword = dfword.drop(dfword[dfword.counts < 100].index)

dfword.head()
dfword.to_csv(r'word_organ.csv', index = False)

In [19]:
dfword.head()

word  counts       date
0  complaint       4 2020-02-01
1  currently       3 2020-02-01
2   underway       3 2020-02-01
3        yet       3 2020-02-01
4  completed       3 2020-02-01

In [16]:
#dfword = pd.DataFrame.from_dict(wordnumbers, orient='index').reset_index()
#dfmain.head()

In [17]:
#dfmain.head()

#dfmain.to_csv(r'data.csv', index = False)